# Supervised Learning para NLP.

El objetivo de la clase de hoy es visitar distintos tipos de problema que nos podemos encontrar en NLP usando supervised Learning.

Habreis visto imágenes similares, pero no tan espectaculares. 
![](https://i.imgur.com/3inJ3wG.jpg)

Supervised Learning es aquel conjunto de algoritmos que precisan de datos etiquetados para aprender. No importa la forma, el algoritmo de aprendizaje, o el de optimización. Si tus datos tienen etiquetas, puedes aplicar supervised learning.

Si en la clase anterior vimos distintos tipos de features, y en Machine Learning y Deep Learning vimos distintos tipos de algoritmos para aplicar, tenemos todas las piezas y  ya estamos listos no? Acordaros, para resolver un problema de supervised learning, siempre buscaremos la misma setup.

1.   Procesar datos. (Big Data) En python podeis usar generators. Si no habéis visto ni uno, avisadme ahora mismo y implementamos uno.
2.   Preparar Xs e Ys.
3.   Escoger algoritmo de entrenamiento.
4.   Entrenar
5.   ??????
6.   Profit

## Que implementaremos desde 0?

Bayes. Veremos Bayes, teoría sobre Naive Bayes algorithm y lo prepararemos todo desde 0. Los demás algoritmos entiendo que los habeís visto en Machine Learning y Deep Learning, y usaremos las librerías de sklearn y Keras como ya hemos visto previamente.

## Objetivos de hoy

* El gran objetivo de hoy, es ver ejemplos de problemas donde podemos aplicar supervised learning. Dejar claro pipelines más o menos estándares de procesado de texto, que algoritmos són más prototípicos en el caso de NLP y entrenar el máximo numero de modelos posibles con datasets reales.

* Veremos problemas de clasificación de varios tipos, desde binarios (me gusta/no me gusta), es decir un sentiment analysis clásico y problemas multiclase, como por ejemplo...(me lo guardo para después). También veremos dos tipos de problemas de clasificación diferente. Veremos un tipo de problema que se llama Modelado d Lenguaje (Language Modeling), y veremos un clásico de NLP como Named Entity Recognition.

* Sobretodo, sobretodo, el objetivo es que no quede duda de que, datos etiquetados = supervised learning, y que para poder entrenar, sólo y únicamente nos hace falta etiquetas, un algoritmo y algo de creatividad.

#### Generators

In [0]:
data = [(x, x) for x in range(50)]

In [0]:
def this_is_not_a_generator(cosas):
    for i, cosa in enumerate(cosas):
        co, sa = cosa
        cosas[i] = (co*2, sa*2)
    return cosas


def this_is_a_generator(cosas):
    for co, sa in cosas:
        yield (co*2, sa*2)

In [0]:
sera_esto_un_generador = this_is_not_a_generator(data)

In [0]:
y_esto = this_is_a_generator(data)

In [9]:
print(type(sera_esto_un_generador)),print(type(y_esto))

<class 'list'>
<class 'generator'>


In [37]:
next(y_esto)

(20, 20)

# Antes de ir a cualquier parte -> Bayes

[Bayes Royal Society](https://www.youtube.com/watch?v=-e8wOcaascM)


<div align="center">
![Bayes](https://static1.squarespace.com/static/591e58f72994cab66b93f891/t/591f89a39c03e001d1e3b17e/1495241602853/bayes-rule-e1350930203949.png =600x)
</div>

Cuando usamos Naive Bayes en NLP? Para casi todo se ha usado Bayes. Los primeros filtros de spam fueron naive bayes classifiers. Casi todos los primeros intentos de solucionar un problema en NLP se sustenta en Bayes.

## Representación de un documento cualquiera en bayes.

![](https://i.imgur.com/3gbPLQY.jpg)

### Calculo de una clase con bayes.

Clasificación con bayes. Probabilidad de que un documento sea de una clase, dado que es esa clase. Hay que representar el documento!

![](https://i.imgur.com/MdJUKgg.jpg)

Alguien puede comentar porque podemos eliminar facilmente el denominador?

### Entreno -> Como estimamos los parametros?

![](https://i.imgur.com/XGJmFwp.jpg)


In [8]:
import numpy as np

Entrenaremos con unigrams por ahora.

In [9]:
datos = [
    ('esto es un ejemplo de lo que podría ser un texto bien', 'ok'),
    ('esto es otro mal ', 'ko'),
    ('me gusta mucho la bien clase KeepCoding', 'ok'),
    ('Este profesor no se mal entera de nada !', 'ko'),
    ('Estoy aprendiendo muchísimo bien !', 'ok'),
    ('me cuesta mucho NLP mal', 'ko'),
    ('NLP es muy fácil si esta bien explicado', 'ok'),
    ('ya falta menos para que mal llegue el break', 'ko'),
    ('Suerte que el profesor nos va a mandar mucho trabajo bien !', 'ok'),
    ('No sé si tendré tiempo a terminar todo el trabajo mal', 'ko'),
        ]

In [10]:
def prepare_data(data):
    data_together = {}
    for x, y in data:
        if y in data_together:
            data_together[y].append(x)
        else: 
            data_together[y]=[x]
    return data_together
    
        
    

In [11]:
from collections import Counter
def get_prior(data):
    prior = Counter([y for _, y in data])
    p = {}
    for pr in prior.keys():
        p[pr]=prior[pr]/len(data)
    return p
    

def get_likelihood(data):
    tokens_in_class = {}
    d = prepare_data(data)
    #cuantas veces w aparece en cada clase
    for label, strings in d.items():
        tokens_in_class[label] = Counter([token for string in strings for token in string.split(" ")])
        print(tokens_in_class[label].most_common(5))
    
    # cuantos tokens por clase
    tokens_per_class = {}
    for label in tokens_in_class.keys():
        tokens_per_class[label] = sum(tokens_in_class[label].values())
    print(tokens_per_class)
    
    likelihood = {}
    for label in tokens_in_class.keys():
        likelihood[label] = {}
        for token in tokens_in_class[label].keys():
            likelihood[label][token] = tokens_in_class[label][token]/tokens_per_class[label]
    return likelihood
    
def train(data):
    likelihood = get_likelihood(data)
    prior = get_prior(data)    
    return likelihood, prior

In [12]:
model = train(datos)

[('bien', 5), ('es', 2), ('un', 2), ('que', 2), ('mucho', 2)]
[('mal', 5), ('el', 2), ('esto', 1), ('es', 1), ('otro', 1)]
{'ok': 44, 'ko': 39}


In [13]:
from math import log2, log, exp

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only difference

## Predicción

¿Probamos con lo que hemos entrenado?

Pero espera! Que es: la primera parte de la ecuación, como la usamos?
¿Qué es exactamente, P(d|c)?

Que es un documento? Un conjunto de features verdad? Además, en este preciso instante es cuando os puedo explicar porque se le llama *Naive* Bayes.

Se le llama así, porque este algoritmo nos obliga a asumir dos cosas muy importantes. Que las features del documento son independientes y que el orden no importa.

Que implica esto? Si el orden no importa podemos usar modelos tipo Bolsas de Palabras (Bag of words).

Que sea independiente, nos permite tratar cada feature con mucha facilidad, si no fuera así, tendríamos que estar calculando cuando afecta cada variable, no a la clasificación del documento, sino entre si.

Acordaros que la assumpción de independencia nos permite relajar la carga computacional, sino sería intratable.

### Dependencia 
<div align="center">
![](https://i.imgur.com/3ARmOZr.jpg =500x)
</div>

### Independencia
<div align="center">
![](https://i.imgur.com/JBn2GhN.jpg =500x)
</div>

Aplicando la asumpción de independencia:
> P(d|c) = P(x_0|c) * ...P(x_i|c)

Última cosa, que usaremos tanto en bayes o language modeling por ejemplo, si tenemos probabilidades muy pequeñas, lo mejor es trabajar en log space, y sumar. Es decir, vamos convirtiendo las probabilidades una a una en log space, sumamos, y luego pasamos de nuevo a probabilidad con la inversa.

In [2]:
0.5*0.3*0.2*0.1

0.003

In [17]:
log_r = log(0.5) + log(0.3) + log(0.2) + log(0.1)
log_r

-5.809142990314028

In [18]:
exp(log_r)

0.002999999999999999

In [6]:
def predict(model, data, alpha=0.00000001):
    likelihood, prior = model
    classes = list(prior.keys())
    print(classes)
    # matriz de predicciones
    predicted = np.zeros((len(data),len(classes)))
    for i, doc in enumerate(data):
        doc_rep = doc.split(" ")
        probs = np.zeros((len(classes)))
        for j, label in enumerate(classes):
            pwc =[log(likelihood[label][x]) if x in likelihood[label] else log(alpha) for x in doc_rep] 
            #sumar en log es igual a multiplicar normal.
            pwc = np.array(pwc)
            prob = np.sum(pwc)
            probs[j] = prob+log(prior[label])
        predicted[i,:] = softmax(probs)
    return np.round(predicted, 2)

In [19]:
xs = [x for x, _ in datos]

In [20]:
predictions = predict(model, xs)
for datap, p in zip(datos, predictions):
    print(datap, p)

['ok', 'ko']
('esto es un ejemplo de lo que podría ser un texto bien', 'ok') [1. 0.]
('esto es otro mal ', 'ko') [0. 1.]
('me gusta mucho la bien clase KeepCoding', 'ok') [1. 0.]
('Este profesor no se mal entera de nada !', 'ko') [0. 1.]
('Estoy aprendiendo muchísimo bien !', 'ok') [1. 0.]
('me cuesta mucho NLP mal', 'ko') [0. 1.]
('NLP es muy fácil si esta bien explicado', 'ok') [1. 0.]
('ya falta menos para que mal llegue el break', 'ko') [0. 1.]
('Suerte que el profesor nos va a mandar mucho trabajo bien !', 'ok') [1. 0.]
('No sé si tendré tiempo a terminar todo el trabajo mal', 'ko') [0. 1.]


In [22]:
datos = ['que pasa si no se nada', 'hablaria de futbol', 'hablar por hablar es tonteria']
predictions = predict(model, datos)
for datap, p in zip(datos, predictions):
    print(datap, p)

['ok', 'ko']
que pasa si no se nada [0. 1.]
hablaria de futbol [0.47 0.53]
hablar por hablar es tonteria [0.64 0.36]


# Recordatorio clase previa

## Pipeline clásico de NLP

![](http://blog.aylien.com/wp-content/uploads/2016/07/nlp-language-dependence-small-1.png)

1. Preprocesado de texto
    1. Limpiar: Caracteres raros, palabras "raras". *Normalizar el texto*
    2. stemmizar?
    3. lemmatizar?
    4. Definir vocabulario?
    5. Tokenizar
    6. Preparar Train/test/val sets
2. Entrenar
3. Testear

### Librerias

In [23]:
%matplotlib inline

import numpy as np
# import spacy

import pickle
import json
import os

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

data_path = '../datasets/data/'

# Preprocesado de texto


datos = json.load('')

datos = pickle.load('')

datos = pymongo.find({}) ... 

Donde tengamos los datos almacenados, sql, lo que sea

El Objectivo principal al finalizar el preprocesado de texto es tener tuplas (o un diccionario, cualquier cosa), que nos permitar representar los datos en formato (x, y, **kwargs). kwargs puede ser cualquier cosa que nos interese para printar por ejemplo, pero al final, x, y como en todos los problemas de supervised learning. No hay ninguna diferencia con imagen o con cualquier otro tipo de datos.


In [27]:
datos = [
    ('esto es un ejemplo de lo que podría ser un texto bien', 'ok'),
    ('esto es otro mal ', 'ko'),
    ('me gusta mucho la bien clase KeepCoding', 'ok'),
    ('Este profesor no se mal entera de nada !', 'ko'),
    ('Estoy aprendiendo muchísimo bien !', 'ok'),
    ('me cuesta mucho NLP mal', 'ko'),
    ('NLP es muy fácil si esta bien explicado', 'ok'),
    ('ya falta menos para que mal llegue el break', 'ko'),
    ('Suerte que el profesor nos va a mandar mucho trabajo bien !', 'ok'),
    ('No sé si tendré tiempo a terminar todo el trabajo mal', 'ko'),
        ]

In [28]:
import string
# from gensim.utils import deaccent
printable_chars = string.printable
printable_chars

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'

In [29]:
datos_w = [(''.join([c if c in printable_chars else '' for c in d[0]]), d[1]) for d in datos]
datos_w

[('esto es un ejemplo de lo que podra ser un texto bien', 'ok'),
 ('esto es otro mal ', 'ko'),
 ('me gusta mucho la bien clase KeepCoding', 'ok'),
 ('Este profesor no se mal entera de nada !', 'ko'),
 ('Estoy aprendiendo muchsimo bien !', 'ok'),
 ('me cuesta mucho NLP mal', 'ko'),
 ('NLP es muy fcil si esta bien explicado', 'ok'),
 ('ya falta menos para que mal llegue el break', 'ko'),
 ('Suerte que el profesor nos va a mandar mucho trabajo bien !', 'ok'),
 ('No s si tendr tiempo a terminar todo el trabajo mal', 'ko')]

## Preparar train/test/val

### Manual

In [31]:
# 1
high = len(datos_w)
test_size = 0.2
size = int(high*test_size)
idx = np.arange(len(datos_w))
print('Todos los elementos: {}'.format(idx))
#random sampling 
test_idx = np.random.choice(idx, size=size, replace=False)
#masking test indices, rest train 
train_mask = np.ones(len(idx), dtype=bool)
train_mask[test_idx] = False
train_idx = idx[train_mask==True]
print('train idx: {}'.format(train_idx))
print('test idx: {}'.format(test_idx))



Todos los elementos: [0 1 2 3 4 5 6 7 8 9]
train idx: [0 1 2 3 4 6 7 9]
test idx: [5 8]


In [32]:
train_data = np.array(datos_w)[train_idx].tolist()
train_data

[['esto es un ejemplo de lo que podra ser un texto bien', 'ok'],
 ['esto es otro mal ', 'ko'],
 ['me gusta mucho la bien clase KeepCoding', 'ok'],
 ['Este profesor no se mal entera de nada !', 'ko'],
 ['Estoy aprendiendo muchsimo bien !', 'ok'],
 ['NLP es muy fcil si esta bien explicado', 'ok'],
 ['ya falta menos para que mal llegue el break', 'ko'],
 ['No s si tendr tiempo a terminar todo el trabajo mal', 'ko']]

In [33]:
test_data = np.array(datos_w)[test_idx].tolist()
test_data

[['me cuesta mucho NLP mal', 'ko'],
 ['Suerte que el profesor nos va a mandar mucho trabajo bien !', 'ok']]

**Alguien nota algun problema con este metodo?**

**Las particiones son equivalentes, mantienen las clases?**

In [34]:
from collections import Counter

In [40]:
label_distribution = Counter([y for _, y in datos_w])
label_train = Counter([y for _, y in train_data])
label_test = Counter([y for _, y in test_data])

s_total = pd.Series([y for _, y in datos_w])
s_train = pd.Series([y for _, y in train_data])
s_test = pd.Series([y for _, y in test_data])

df = pd.DataFrame([s_total, s_train, s_test]).T
df.rename(index=str, columns={0: 'total', 1: 'train', 2:'test'}, inplace=True)
df


,total,train,test
0,ok,ok,ko
1,ko,ko,ok
2,ok,ok,NaN
3,ko,ko,NaN
4,ok,ok,NaN
5,ko,ok,NaN
6,ok,ko,NaN
7,ko,ko,NaN
8,ok,NaN,NaN
9,ko,NaN,NaN


In [37]:
from sklearn.model_selection import StratifiedShuffleSplit

In [38]:
def split_train_val_test(dataset, split=0.2):

    x, y = zip(*dataset)
    x = np.array(list(x))
    y = np.array(list(y))
    sss = StratifiedShuffleSplit(n_splits=1, test_size=split, random_state=1337) #l33t seed
    for train_index, test_index in sss.split(x, y):
        x_train, x_val = x[train_index], x[test_index]
        y_train, y_val = y[train_index], y[test_index]
    splits = {'train':(x_train, y_train), 'test':(x_val, y_val)}
    return splits

In [41]:
split = split_train_val_test(datos_w)
split

{'test': (array(['Estoy aprendiendo muchsimo bien !',
         'No s si tendr tiempo a terminar todo el trabajo mal'],
        dtype='<U59'), array(['ok', 'ko'], dtype='<U2')),
 'train': (array(['esto es un ejemplo de lo que podra ser un texto bien',
         'me gusta mucho la bien clase KeepCoding', 'esto es otro mal ',
         'NLP es muy fcil si esta bien explicado',
         'Suerte que el profesor nos va a mandar mucho trabajo bien !',
         'Este profesor no se mal entera de nada !',
         'me cuesta mucho NLP mal',
         'ya falta menos para que mal llegue el break'], dtype='<U59'),
  array(['ok', 'ok', 'ko', 'ok', 'ok', 'ko', 'ko', 'ko'], dtype='<U2'))}

# Entrenar

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [0]:
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__ngram_range': ((1, 1), (1, 2), (1, 3)),  # unigrams or bigrams
}

In [0]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)

In [114]:
grid_search.fit(split['train'][0], split['train'][1])
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))
print("Best score: %0.3f" % grid_search.best_score_)

Fitting 3 folds for each of 36 candidates, totalling 108 fits
	tfidf__norm: 'l1'
	tfidf__use_idf: False
	vect__max_df: 0.5
	vect__ngram_range: (1, 1)
Best score: 0.875


[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:    0.3s finished


In [115]:
pipeline.set_params(**best_parameters)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...alse,
         use_idf=False)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [0]:
# pipeline.steps[0][1].vocabulary_

# Testear

In [116]:
predictions = pipeline.score(split['test'][0], split['test'][1])
print('TEST SCORE: {}'.format(predictions))

TEST SCORE: 1.0


# Jugar / Deploy

In [0]:
predict_sentences = [
    'que bien hay clase',
    'que mal hay clase',
    'que mal el profesor',
    'que mal dataset construyo',
    'que bien hoy no vemos el futbol'
]

In [123]:
predictions = pipeline.predict(predict_sentences)
predictions_probs = pipeline.predict_proba(predict_sentences)
prediction_hand = predict(model, predict_sentences)
for x, y, p, ph in zip(predict_sentences, predictions, predictions_probs, prediction_hand):
    print('{} ------> {} {} {}'.format(x, y, p, ph)) # p

['ok', 'ko']
que bien hay clase ------> ok [0.45273402 0.54726598] [1. 0.]
que mal hay clase ------> ko [0.5271332 0.4728668] [0.16 0.84]
que mal el profesor ------> ko [0.53070962 0.46929038] [0. 1.]
que mal dataset construyo ------> ko [0.55718 0.44282] [0. 1.]
que bien hoy no vemos el futbol ------> ok [0.48078911 0.51921089] [0.86 0.14]
